# Kickstarter Startup Analysis

1. How should we design our Kickstarter campaign in order to make our startup funding phase as succesful as possible?
    - **Categorie** (Assumption Tech) - Succesful campaigns
        -> ratio succesful/unsuccesful/live definieren
        -> one-sample 1 tegenover het totaal
        -> two sample 1 tegenover de rest, of Tech vs Design
    - **US vs NL** (Assumption: Silicon Valley vs. the Netherlands)
        -> Average pledged amount in NL hoger OF lager dan in US (1 of 2 sample T test)
    - **Number of backers** (Assumption: popularity of campaign defines popularity of startup)
        -> Ga je voor weinig backers met veel of veel backers met weinig -> verschil in succesful (Hoeveel backers per in je goal)
    -> Zoja/Zonee -> define confidence intervals voor backers voor ons "start-up" plan
    
    - **Pledge amount asked** - Pledged (goal - converted)
    
2. Which pledge amount will lead to the highest possible funding?

3. How could the location of the kickstarter campaign, or the sub-category, influence the success rate of our kickstarter campaign?

**To do**  
~~Elske: Fix git issues  ~~  
Rianne: #Drop duplicates (per project_id, check categorieen - 1, 2 en 3 -- zijn hetzelfde, dus sort_values by pledged amount, keepfirst)  
Rianne: #Currency -> use usd_pledged -> convert_amount can be dropped, fx_rate also, currency also   
Rianne: #Take out live  
~~Elske:  #Canceled and unsuccesful -> Failed~~  
~~Elske:  #Unfold Dict "location" unfold: 'name', 'expanded_country'~~  
~~Elske:  #Create column with average pledge per backer  ~~  
Both: Check for missing values and how to solve them + add code for hypothesis testing

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import json
from datetime import datetime
import scipy.stats as stats
from scipy.stats import norm

Download the file <a href="https://drive.google.com/drive/folders/1hIBnfoXrXNW2vF_ka8R8Y0mltCmBjqPZ" target="_blank">here</a>.

In [2]:
#Importing the data using a relative path
#Mac
#df = pd.read_csv('../Kickstarter_analysis/sep2020.csv', converters={'category': eval})
#df.head()

#Windows
df = pd.read_csv('.\\data\\sep2020.csv')
df.head()

,backers_count,blurb,country,created_at,deadline,goal,id,launched_at,name,pledged,spotlight,state,state_changed_at,usd_pledged,category_id,category_name,category_parent_id,category_parent_name,project_id
0,61,Support great art! Join us as we re-stage the ...,US,1579292017,1583025192,3000.0,1568973024,1580433192,Revisionist History 2,4618.0,True,successful,1583025192,4618.0,254,Performances,6.0,Dance,3914215
1,52,JinBucha is a new kind of Brewery in North Par...,US,1446051515,1450118057,20000.0,1782378629,1447526057,JinBucha: A Modern Kombucha tasting room in No...,3461.0,False,failed,1450118057,3461.0,307,Drinks,10.0,Food,2198010
2,0,"""Taste The Scent of Tea"" Rose and Blanc Tea Ro...",US,1518159717,1520233260,15000.0,2076142479,1518208887,Tea Room Cafe - Daily & Herb Tea,0.0,False,failed,1520233260,0.0,307,Drinks,10.0,Food,3304826
3,2,Bringing Philly cheesesteaks (and other delici...,US,1454217596,1459885844,2000.0,1416797221,1454705444,HELP TWO BROTHERS OPEN A FOOD TRUCK!,25.0,False,failed,1459885844,25.0,311,Food Trucks,10.0,Food,2349147
4,150,a frog plushie keychain and frog butt pin for ...,US,1571342290,1574445600,800.0,1708708142,1573236000,Son the Frog Plushie Keychain + JUICY Pin,3275.0,True,successful,1574445600,3275.0,28,Product Design,7.0,Design,3834865


**Step 1:** Data Cleaning and Wrangling
- Transforming unix time stamps to datetime

In [3]:
# Dates transformed [['created_at'], ['deadline'], ['launched_at'], ['state_changed_at']]
df[['created_at','deadline', 'launched_at', 'state_changed_at']].apply(pd.to_datetime, unit='s')

,created_at,deadline,launched_at,state_changed_at
0,2020-01-17 20:13:37,2020-03-01 01:13:12,2020-01-31 01:13:12,2020-03-01 01:13:12
1,2015-10-28 16:58:35,2015-12-14 18:34:17,2015-11-14 18:34:17,2015-12-14 18:34:17
2,2018-02-09 07:01:57,2018-03-05 07:01:00,2018-02-09 20:41:27,2018-03-05 07:01:00
3,2016-01-31 05:19:56,2016-04-05 19:50:44,2016-02-05 20:50:44,2016-04-05 19:50:44
4,2019-10-17 19:58:10,2019-11-22 18:00:00,2019-11-08 18:00:00,2019-11-22 18:00:00
...,...,...,...,...
219683,2019-02-21 18:33:41,2019-03-29 21:00:00,2019-02-27 15:12:10,2019-03-29 21:00:02
219684,2013-03-05 20:13:03,2013-08-10 07:45:40,2013-07-05 07:45:40,2013-08-10 07:45:40
219685,2016-06-21 00:35:01,2016-08-13 21:49:58,2016-07-14 21:49:58,2016-08-13 21:49:58
219686,2015-11-28 19:14:15,2016-01-09 19:19:33,2015-12-10 19:19:33,2016-01-09 19:19:34


- Remove live kickstarter campaings.

In [4]:
# Drop records of unfinished project (category live)
df = df.loc[df['state'] != 'live']
len(df)

# Check if records are dropped
df['state'].value_counts()

successful    128156
failed         76137
canceled        9138
Name: state, dtype: int64

- Drop duplicated projects

In [5]:
# Print number of records
print('number of records: ', len(df))

# sort by highest pledged and drop duplicated project ids
df = df.sort_values(by='usd_pledged',ascending=False).drop_duplicates(subset='project_id', keep='first')

# check number of records
print('number of records after dropping duplicated project ids: ', len(df))

number of records:  213431
number of records after dropping duplicated project ids:  188605


- Classify cancelled and unsuccesful as both "unsuccesful"  
Since we are looking to launch a succesful campaign for our start-up, 
we categorize the data in two categories: "Succesful" and "Failed" (compromising canceled and failed campaings).

In [6]:
#Value_counts shows there are both canceled and failed campaigns. Categorize into successfull and unsuccessful
df['state_cat'] = np.where(df['state']=='successful', 'successful', 'unsuccessful')
df['state_cat'].value_counts()

successful      104210
unsuccessful     84395
Name: state_cat, dtype: int64

- Create variable 'average_pledge' per backer

In [7]:
df['average_pledge'] = (df['usd_pledged']/df['backers_count'])
df['average_pledge'].head()

26699      155.897828
155651     128.089024
99013      135.556872
119358    1062.651186
201818     227.970892
Name: average_pledge, dtype: float64

- Check for missing values

In [8]:
df.isna().sum()

backers_count               0
blurb                       8
country                     0
created_at                  0
deadline                    0
goal                        0
id                          0
launched_at                 0
name                        0
pledged                     0
spotlight                   0
state                       0
state_changed_at            0
usd_pledged                 0
category_id                 0
category_name               0
category_parent_id       7737
category_parent_name     7737
project_id                  0
state_cat                   0
average_pledge          16009
dtype: int64

# Data Exploration

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [10]:
#Grouping the df by main category, sorted by sum of converted_pledged amount.
top_pledged = df.loc[:,df['state'] == 'succesful'].groupby(['category_parent_name']).sum().reset_index().sort_values(by='converted_pledged_amount', ascending=False)
top_backers = df.groupby(['category_parent_name']).sum().reset_index().sort_values(by='backers_count', ascending=False)
#Selecting top five from top_pledged and top_backers
top_five_pledged = top_pledged.head(5)
top_five_backers = top_backers.head(5)
pal_pled = sns.color_palette("Greens_d", len(top_five_pledged))
pal_back = sns.color_palette("Reds_d", len(top_five_backers))
rank_pled = top_five_pledged["converted_pledged_amount"].argsort().argsort()
rank_back = top_five_backers["backers_count"].argsort().argsort()
#Plotting the top five
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 4))
sns.barplot(ax =axes[0], x='converted_pledged_amount',y='category_parent_name', data=top_five_pledged, linewidth=0.7, palette=np.array(pal_pled[::+1])[rank_pled])
axes[0].set_title('Converted Amount Pledged')
sns.barplot(ax=axes[1], x='backers_count',y='category_parent_name', data=top_five_backers, linewidth=0.7, palette=np.array(pal_back[::+1])[rank_back])
axes[1].set_title('Number of Backers');

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
plt.figure(figsize=(15, 10))
sns.boxplot(x='category_parent_name', y='converted_pledged_amount', hue='state', data=df, orient='v', showfliers=False);
plt.title('Converted Pledged Amount per Category Parent', fontsize=20);
plt.xlabel('Category Parent', fontsize=18)
plt.ylabel('Converted Pledged Amount)', fontsize=18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=15);

#Can we use "proportion of the population" - om significant verschil tussen succesful (tech en de rest)


In [ ]:
df.groupby(['category_name']).sum().sort_values(by='converted_pledged_amount', ascending=False)

In [ ]:
#corr
corr = df['converted_pledged_amount'].corr(df['backers_count'])
#set figure size
plt.figure(figsize=(15, 10))
#plot the data
sns.regplot(data=df,x=df['converted_pledged_amount'],y=df['backers_count'], ci=None)
#format the plot
text = 'r=%s' % (corr)
plt.text(1, 19, text, fontsize=14)
plt.title('', fontsize=20);
plt.xlabel('', fontsize=18)
plt.ylabel('', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18);

# Statistical Analysis

In [ ]:
#Confidence Intervals:
# To define between pledge amount x and x we should ask
# To define between number of backers x and x we should aim for

# One-sample
# 1 category opposed to population mean

# Two-sample
#2 categories (tech vs. food) and Netherlands/Europe vs. US

**Categories** (Assumption that Technology startups have highest funding due to popularity/size of investments) 

https://producthype.co/kickstarter-tips-2019/ 

        - Succesful campaigns
        -> ratio succesful/unsuccesful/live definieren
        -> one-sample 1 tegenover het totaal
        -> two sample 1 tegenover de rest, of Tech vs Design 
        
*   H0: $\mu_{1} = \mu_{2}$, There is no difference between total pledge for successful campaigns in the Technology category and all other categories
*   H1: $\mu_{1} \neq \mu_{2},$ There is a difference between total pledge for successful campaigns in the Technology category and all other categories

For this test, we have to groups in which we can not match the rows to eachother, hence **independent samples**.
Secondly, we assume a **normally distributed population**.

(?) We cannot (or do not want to) assume Equal Variances.

In [ ]:
#Compute variables to test
succesful_campaigns = df.loc[df['state'] == 'successful']
succesful_campaigns_tech = succesful_campaigns.loc[succesful_campaigns['category_parent_name'] == 'Technology']
succesful_campaigns_other = succesful_campaigns.loc[succesful_campaigns['category_parent_name'] != 'Technology']

#Plot to estimate differences
succesful_campaigns['tech_vs_other'] = np.where(succesful_campaigns['category_parent_name']=='Technology', 'Technology', 'Other')
ax = sns.boxplot(x="tech_vs_other", y="usd_pledged", data=succesful_campaigns, order=["Technology", "Other"], showfliers=False)

In [ ]:
#Plot boxplots versus other five top categories according to count
ax = sns.boxplot(x="category_parent_name", y="usd_pledged", data=succesful_campaigns, order=["Technology", "Design", "Games", "Food", "Fashion"], showfliers=False)

In [ ]:
from scipy.stats import ttest_ind

#Decide whether equal variances are true or false
ttest_ind(succesful_campaigns_tech['usd_pledged'], succesful_campaigns_other['usd_pledged'], equal_var=False)
ttest_ind(succesful_campaigns_tech['usd_pledged'], succesful_campaigns_other['usd_pledged'], equal_var=True)

In [ ]:
#Idea: build a loop two-sample-t-test for testing Technology against Design, Games, Food and Fashion.

- US vs NL (Assumption: Silicon Valley vs. the Netherlands)
    - Usd pledged in NL hoger OF lager dan in US (1 of 2 sample T test)  
    -> Dict "location" needs to be unfolded ('name', 'expanded_country')  
    -> Test binnen alle categorien eerst, eventueel later alleen in hierboven gekozen categorie  
Rianne

In [ ]:
# Create two samples: one US and one NL 
df_us = df.loc[df['country']=='US']
df_nl = df.loc[df['country']=='NL']
df_usnl = df[((df['country'] == 'US') | (df['country'] == 'NL'))]
print('Number of records US: ', len(df_us))
print('Number of records NL: ', len(df_nl))
print('Number of records Us & NL: ', len(df_usnl))

In [ ]:
df_nl['state'].value_counts()

In [ ]:
#Import libraries

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import chi2_contingency

#contingency table
contingency_table = pd.crosstab(df_usnl['state_cat'], df_usnl['country'])
contingency_table

#perform test
stats.chi2_contingency(contingency_table)

"""Because the p-value is smaller than 0.05, we can reject the 0 hypothesis. 
H0 = no statistical difference, that means there is a statistical difference. The chance to have a succesfull campaign in the US
is higher than in the Neterlands."""

- Number of backers (Assumption: popularity of campaign defines popularity of startup)
    - Ga je voor weinig backers met veel of veel backers met weinig -> verschil in succesful  
        -> Define average pledge per backer (see to do)  
        -> Succesful versus unsuccesful (2-sample: avg. per backer succesful versus avg. per backer unsuccesful)  
            We verwachten dat dit verschilt - gemiddelde bij succesvol hoger
Rianne  
            
- Zoja/Zonee -> define confidence intervals voor backers voor ons "start-up" plan (binnen onze categorie?)  

Elske

In [ ]:
# Sample represents the numbers of backers for succesful campaigns within the "Tech" category
succesful_campaigns = df.loc[df['state'] == 'successful']
succesful_campaigns_tech = succesful_campaigns.loc[succesful_campaigns['category_parent_name'] == 'Technology']
sample = succesful_campaigns_tech['backers_count']

#Compute degrees of freedom
df = len(sample) -1 

#Compute t-statistic, mean and standard error
t = stats.t.interval(0.95,df)[1]
x  = np.mean(sample)
se = np.std(sample) / np.sqrt(len(sample))

#Define confidence intervals
con_int = [round(num,3) for num in [x - (t * se) , x + (t * se)]]
print('With 95 % confidence, the average number of backers for a succesful campaign in Technology:', round(con_int[0]), "and", round(con_int[1]))

#### Confidence interval amount pledged

In [ ]:
# Sample represents the pledged US dollars for succesful campaigns within the “Tech” category
succesful_campaigns = df.loc[df['state'] == 'successful']
succesful_campaigns_tech = succesful_campaigns.loc[succesful_campaigns['category_parent_name'] == 'Technology']
sample = succesful_campaigns_tech['usd_pledged']

#Compute degrees of freedom
dfree = len(sample) -1

#Compute t-statistic, mean and standard error
t = stats.t.interval(0.95,dfree)[1]
x  = np.mean(sample)
se = np.std(sample) / np.sqrt(len(sample))

#Define confidence intervals
con_int = [round(num,3) for num in [x - (t * se) , x + (t * se)]]
print('With 95 % confidence, the average usd_pledged for a succesful campaign in Technology is between:', round(con_int[0]), 'and', round(con_int[1]), 'dollar')

#### Confidence interval successful goal

In [11]:
# Sample represents the numbers of backers for succesful campaigns within the “Tech” category
sample = succesful_campaigns_tech['goal']

#Compute degrees of freedom
dfree = len(sample) -1

#Compute t-statistic, mean and standard error
t = stats.t.interval(0.95,dfree)[1]
x  = np.mean(sample)
se = np.std(sample) / np.sqrt(len(sample))

#Define confidence intervals
con_int = [round(num,3) for num in [x - (t * se) , x + (t * se)]]
print('With 95 % confidence, the average goal for a succesful campaign in Technology is between:', round(con_int[0]), 'and', round(con_int[1]), 'dollar')

NameError: name 'succesful_campaigns_tech' is not defined

#### Goal differs significantly from pledged amout

In [ ]:
#H0: mean goal = mean usd_pledged
#HA: mean goal != mean usd_pledged

from scipy.stats import ttest_rel
result = ttest_rel(succesful_campaigns_tech['usd_pledged'], succesful_campaigns_tech['goal'])
pvalue = result.pvalue
pvalue

if pvalue < 0.05:
    print('Reject null hypothesis: the mean of the goal and pledged amout significantly differ')
    
    if pvalue > 0.05:
        print('Confirm null hypothesis: the mean of the goal and pledged amout do not significantly differ')

Bonus:
In specific categories: Pledge amount asked - Pledged (goal - converted)  
COVID impact 2020 versus 2018/2019 (less pledges than previous years - and in which categories?)  
Other differences in years   
    